In [28]:
import pandas as pd
from sodapy import Socrata

This checkout titles class utilizes the Socrata API to access the Seattle Public Library Checkouts by Title dataset.
It even has the option to query the database to grab a subset of data such as only certain months or years or checkouts for books only. After loading the subset of the database, it returns the head of the pandas dataframe. Finally, you have the option to save the subset of data to a csv file locally on your machine.

In [42]:
class checkout_titles:
    def __init__(self, limit=1000, datasource="data.seattle.gov", ID="tmmm-ytt6", results_df=None):
        '''Default to call the Seattle Public Library Checkouts
        dataset, but it can call other databases via socrata.'''
        self.limit = limit
        self.datasource = datasource
        self.ID = ID
        self.results_df= results_df
    def load_subset(self, limit, query=False):
        '''Uses the sodapy Socrata as an API to access the 
            Seattle Public Library checkouts dataset.'''
        client = Socrata(self.datasource, None)
        if query is not False:
            try:
                results = client.get(self.ID, query=query)
            except HTTPError:
                print('For queries, terms, such as select, groupby, or where,',
                      'need to be lowercase. Strings need to be in quotes',
                      'e.g. "EBOOK" and numbers need to not have quotes.')
            self.results_df = pd.DataFrame.from_records(results)
        else:
            results = client.get("tmmm-ytt6", limit=limit)
            self.results_df = pd.DataFrame.from_records(results)
        return self.results_df.head()
    
    def save_subset(self, filepath):
        '''Saves the subset of data loaded from the Seattle Public Library Checkouts dataset as csv.'''
        self.results_df.to_csv(filepath)

Here is an example of using the class in action!

In [43]:
spl = checkout_titles()

In [44]:
spl.load_subset(limit=1000)

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,creator,subjects,publisher,publicationyear
0,Physical,Horizon,BOOK,2007,12,35,Richistan : a journey through the American wea...,"Frank, Robert, 1968-","Wealth United States, Rich people United State...","Crown Publishers,",c2007.
1,Physical,Horizon,BOOK,2007,12,2,Wire knits / Heather Kingsley-Heath.,"Kingsley-Heath, Heather","Wire craft, Knitting","Search,",2007.
2,Physical,Horizon,SOUNDDISC,2007,12,15,audiences listening,NaN,"House music, Popular music 2001 2010",NaN,NaN
3,Physical,Horizon,BOOK,2007,12,1,African ark : people and ancient cultures of E...,"Beckwith, Carol, 1945-","Ethnology Africa Northeast, Africa Northeast S...","H.N. Abrams,",1990.
4,Physical,Horizon,VIDEODISC,2007,12,8,Superman the animated series Volume two The ne...,NaN,"Video recordings for the hearing impaired, Ani...",NaN,NaN


In [45]:
spl.save_subset('/home/jupyter/spl_checkouts2.csv')

Below is an example of using a query.

In [47]:
spl2 = checkout_titles()

In [48]:
spl2.load_subset(limit=1000, query='select * where checkoutyear > 2015')

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,creator,subjects,publisher,publicationyear
0,Digital,OverDrive,EBOOK,2016,10,1,Slim by Design: Mindless Eating Solutions for ...,Brian Wansink,"Health & Fitness, Nonfiction, Sociology",HarperCollins Publishers Inc.,2014
1,Physical,Horizon,VIDEODISC,2016,10,7,Trollflöjten [videorecording] = The magic flut...,NaN,"Operas, Feature films","Criterion Collection,",[2000]
2,Digital,OverDrive,EBOOK,2016,10,1,Great Expectations,Charles Dickens,"Classic Literature, Fiction","Penguin Group (USA), Inc.",2011
3,Physical,Horizon,BOOK,2016,10,1,A dangerous friend / Ward Just.,"Just, Ward S.","Vietnam War 1961 1975 Fiction, Vietnam History...","Houghton Mifflin Co.,",1999.
4,Physical,Horizon,SOUNDDISC,2016,10,3,Bridge School benefit [sound recording].,NaN,"Rock music 2011 2020, Popular music 2011 2020","Reprise,",p2011.


In [49]:
spl2.save_subset('/home/jupyter/spl_checkouts3.csv')

Once the CSV file is saved, it is then uploaded into Tableau for some data-vis. Here is the [Tableau workbook](https://public.tableau.com/app/profile/louisa.reilly/viz/spl_checkout_subset/checkouts_per_year?publish=yes). The workbook is still in the development process, and there are plans to build an interactive dashboard.

The Checkouts by Title dataset is updated monthly on the 6th. It can be updated by doing another query for that month, and appending it to that dataset. Some next steps are to add an update function to the checkout_title class which will obtain the most recent month of data and append it to the existing local corpus. The corpus CSV file will then be uploaded back into the Tableau workbook, so each visualization can have the additional datapoints.